In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

In [4]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [5]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [4]:
relevantAttributes = ['totals.hits', 'visitNumber', 'totals.pageviews',
                      'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.city',
                      'device.operatingSystem', 'device.browser',
                      'channelGrouping',
                      'totals.transactionRevenue']

train_data['totals.transactionRevenue'] = train_data['totals.transactionRevenue'].fillna(0)
train_data['totals.pageviews'] = train_data['totals.pageviews'].fillna(1)

trainData = train_data.sample(frac=0.8,random_state=200)
testData = train_data.drop(trainData.index)

trainData_PM = trainData[relevantAttributes]
testData_PM = testData[relevantAttributes]

trainData_PM = pd.DataFrame(trainData_PM)
testData_PM = pd.DataFrame(testData_PM)

In [5]:
#Figure out labeling for country param
sortedContinent = train_data.groupby('geoNetwork.continent')['totals.transactionRevenue'].mean()
sortedContinent = sortedContinent.sort_values(ascending=False)
continentCodeDict = {}
i = 0
for continent in sortedContinent.index:
    continentCodeDict[continent] = i
    i += 1
    
sortedCountries = train_data.groupby('geoNetwork.country')['totals.transactionRevenue'].mean()
sortedCountries = sortedCountries.sort_values(ascending=False)
countryCodeDict = {}
i = 0
for country in sortedCountries.index:
    countryCodeDict[country] = i
    i += 1
    
sortedCities = train_data.groupby('geoNetwork.city')['totals.transactionRevenue'].mean()
sortedCities = sortedCities.sort_values(ascending=False)
cityCodeDict = {}
i = 0
for city in sortedCities.index:
    cityCodeDict[city] = i
    i += 1
    
sortedDevices = train_data.groupby('device.operatingSystem')['totals.transactionRevenue'].mean()
sortedDevices = sortedDevices.sort_values(ascending=False)
deviceCodeDict = {}
i = 0
for device in sortedDevices.index:
    deviceCodeDict[device] = i
    i += 1
    
sortedBrowser = train_data.groupby('device.browser')['totals.transactionRevenue'].mean()
sortedBrowser = sortedBrowser.sort_values(ascending=False)
browserCodeDict = {}
i = 0
for browser in sortedBrowser.index:
    browserCodeDict[browser] = i
    i += 1

sortedChannelGrouping = train_data.groupby('channelGrouping')['totals.transactionRevenue'].mean()
sortedChannelGrouping = sortedChannelGrouping.sort_values(ascending=False)
channelGroupingCodeDict = {}
i = 0
for device in sortedChannelGrouping.index:
    channelGroupingCodeDict[device] = i
    i += 1

print(channelGroupingCodeDict)
print(deviceCodeDict)
print(browserCodeDict)
print(continentCodeDict)
#print(cityCodeDict)

{'Display': 0, 'Referral': 1, 'Direct': 2, 'Paid Search': 3, 'Organic Search': 4, '(Other)': 5, 'Affiliates': 6, 'Social': 7}
{'Chrome OS': 0, 'Macintosh': 1, 'Linux': 2, 'Windows': 3, 'Android': 4, 'iOS': 5, 'Windows Phone': 6, 'NTT DoCoMo': 7, 'BlackBerry': 8, 'Firefox OS': 9, 'FreeBSD': 10, 'Nintendo 3DS': 11, 'Xbox': 12, 'Nintendo Wii': 13, 'Nintendo WiiU': 14, 'Nokia': 15, 'OpenBSD': 16, 'Samsung': 17, 'SunOS': 18, '(not set)': 19}
{'Firefox': 0, 'Chrome': 1, 'Edge': 2, 'Internet Explorer': 3, 'Safari': 4, 'Amazon Silk': 5, 'Opera': 6, 'Safari (in-app)': 7, 'Android Webview': 8, 'DASH_JR_3G': 9, 'Konqueror': 10, 'Iron': 11, 'IE with Chrome Frame': 12, 'Hisense M20-M_LTE': 13, 'HTC802t_TD': 14, 'DoCoMo': 15, 'subjectAgent: NoticiasBoom': 16, 'Coc Coc': 17, 'LYF_LS_4002_12': 18, 'Changa 99695759': 19, 'CSM Click': 20, 'BlackBerry': 21, 'Apple-iPhone7C2': 22, 'Android Runtime': 23, 'Android Browser': 24, 'ADM': 25, '0': 26, 'LYF_LS_4002_11': 27, 'M5': 28, 'Lunascape': 29, 'Reddit': 3

In [6]:
trainY = trainData_PM["totals.transactionRevenue"]
trainX = trainData_PM[['totals.hits', 'visitNumber', 'totals.pageviews',
                        'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.city', 
                        'device.operatingSystem', 'device.browser',
                        'channelGrouping']]

trainX['geoNetwork.continent'] = trainX['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
trainX['geoNetwork.country'] = trainX['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
trainX['geoNetwork.city'] = trainX['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
trainX['device.operatingSystem'] = trainX['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
trainX['device.browser'] = trainX['device.browser'].transform(lambda x: browserCodeDict[x])
trainX['channelGrouping'] = trainX['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

testY = testData_PM["totals.transactionRevenue"]
testX = testData_PM[['totals.hits', 'visitNumber', 'totals.pageviews',
                      'geoNetwork.continent','geoNetwork.country', 'geoNetwork.city',
                            'device.operatingSystem', 'device.browser',
                      'channelGrouping']]

testX['geoNetwork.continent'] = testX['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
testX['geoNetwork.country'] = testX['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
testX['geoNetwork.city'] = testX['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
testX['device.operatingSystem'] = testX['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
testX['device.browser'] = testX['device.browser'].transform(lambda x: browserCodeDict[x])
testX['channelGrouping'] = testX['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])


/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [7]:
test_data = pd.read_csv('test_flat.csv', dtype={'fullVisitorId': 'str', 'visitId': 'str'}) # Important!!

test_data['pageviews'] = test_data['pageviews'].fillna(1)

X = test_data[['hits', 'visitNumber', 'pageviews', 
               'continent', 'country', 'city',
               'operatingSystem', 'browser', 'channelGrouping']]

X['continent'] = X['continent'].transform(lambda x: continentCodeDict[x])
X['country'] = X['country'].transform(lambda x: countryCodeDict.get(x,220))
X['city'] = X['city'].transform(lambda x: cityCodeDict.get(x, (len(test_data['city']) -2)))
X['operatingSystem'] = X['operatingSystem'].transform(lambda x: deviceCodeDict.get(x,20))
X['browser'] = X['browser'].transform(lambda x: browserCodeDict.get(x, (len(test_data['browser'])-2)))
X['channelGrouping'] = X['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [8]:
X_train, X_val, y_train, y_val = train_test_split(trainX, trainY, test_size=0.15, random_state=1)

print("Train shape:", X_train.shape)
print("Validation shape: ", X_val.shape)
print("Test (submit) shape: ", X.shape)

Train shape: (614483, 9)
Validation shape:  (108439, 9)
Test (submit) shape:  (804684, 9)


In [9]:
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

In [10]:
def run_lgb(X_train, y_train, X_val, y_val, X_test):
    
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgb_train_data = lgb.Dataset(X_train, label=y_train)
    lgb_val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(params, lgb_train_data, 
                      num_boost_round=5000,
                      valid_sets=[lgb_train_data, lgb_val_data],
                      early_stopping_rounds=100,
                      verbose_eval=500)

    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
    y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred_submit = model.predict(X_test, num_iteration=model.best_iteration)

    print(f"LGBM: RMSE val: {rmse(y_val, y_pred_val)}  - RMSE train: {rmse(y_train, y_pred_train)}")
    return y_pred_submit, model

lgb_preds, lgb_model = run_lgb(X_train, y_train, X_val, y_val, X)
print(lgb_preds)

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[94]	training's rmse: 5.58403e+07	valid_1's rmse: 2.61428e+07
LGBM: RMSE val: 26142752.28441  - RMSE train: 55840307.98551
[1128732.52452256 1112414.46186534 1112414.46186534 ... 5516684.82960697
 3887852.67330654 3887852.67330654]


In [8]:
# sc = StandardScaler()
# trainX = sc.fit_transform(trainX)
# testX = sc.transform(testX)

In [15]:
# model = XGBRegressor(n_estimators=100,
#                     learning_rate = .1,
#                     max_depth = 6,
#                     random_state=42,
#                     n_jobs = -1,
#                     early_stopping_rounds=10)


# model.fit(
#     trainX, 
#     trainY, 
#     eval_metric="rmse",
#     eval_set=[(testX, testY)],
#     verbose=True, )

# from sklearn.linear_model import LinearRegression
# from sklearn import cross_validation, tree, linear_model
# regr = linear_model.LinearRegression()

# regr.fit(trainX, trainY)

/home/tintash/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
y_pred=cat_model.predict(testX)

y = testY

# y_pred = y_pred + 1
# y = y + 1

y_pred_ln = np.log1p(y_pred)
y_true_ln = np.log1p(y)

difference = np.subtract(y_pred_ln, y_true_ln)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log1p
  


12.913862717860999

**Light GB**

In [15]:
def run_catboost(X_train, y_train, X_val, y_val, X_test):
    model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
    model.fit(X_train, y_train,
              eval_set=(X_val, y_val),
              use_best_model=True)
    
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
#     y_pred_submit = model.predict(X_test)

    print(f"CatB: RMSE val: {rmse(y_val, y_pred_val)}  - RMSE train: {rmse(y_train, y_pred_train)}")
    return  model

cat_model = run_catboost(X_train, y_train, X_val, y_val, X)

0:	learn: 57408882.0081824	test: 26791770.6716817	best: 26791770.6716817 (0)	total: 148ms	remaining: 2m 27s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 26079788.41
bestIteration = 16

Shrink model to first 17 iterations.
CatB: RMSE val: 26079788.41714  - RMSE train: 52392263.65598


In [12]:
# Note: this is currently being reconstructed!
ensemble_preds_70_30_00 = 0.7 * lgb_preds + 0.3 * cat_preds 
ensemble_preds_70_25_05 = 0.7 * lgb_preds + 0.25 * cat_preds

In [13]:
def submit(predictions, filename='test_flat.csv'):
    """
    Takes a (804684,) 1d-array of predictions and generates a submission file named filename
    """
    

    y_pred = lgb_model.predict(X  , num_iteration=lgb_model.best_iteration)

    y_pred += 1

    test_data['predicted'] = y_pred
    # test_data[['fullVisitorId', 'predicted']].to_csv('output_row.csv')

    y_pred_sum = test_data.groupby('fullVisitorId')['predicted'].sum()

    y_pred_log = np.log(y_pred_sum)

    #y_pred_log.to_csv('output_log.csv')

    print(y_pred_log)

    # y_pred_sum.to_csv('output_sum.csv')

submit(lgb_preds, "submit-lgb.csv")
# Note: I disabled XGB to make the notebook run faster
submit(cat_preds, "submit-cat.csv")
submit(ensemble_preds_70_30_00, "submit-ensemble-70_30_00.csv")
submit(ensemble_preds_70_25_05, "submit-ensemble-70_25_05.csv")

ensemble_preds_70_30_00_pos = np.where(ensemble_preds_70_30_00 < 0, 0, ensemble_preds_70_30_00)
submit(ensemble_preds_70_30_00_pos, "submit-ensemble-70_30_00-positive.csv")

ensemble_preds_70_25_05_pos = np.where(ensemble_preds_70_25_05 < 0, 0, ensemble_preds_70_25_05)
submit(ensemble_preds_70_25_05_pos, "submit-ensemble-70_25_05-positive.csv")

NameError: name 'load_preprocessed_dfs' is not defined